In [1]:
# Run some setup code for this notebook.

import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'

# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

In [3]:
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training data shape:  (50000, 32, 32, 3)
Training labels shape:  (50000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [4]:
# Subsample the data for more efficient code execution in this exercise
num_training = 5000
mask = list(range(num_training))
X_train = X_train[mask]
y_train = y_train[mask]

num_test = 500
mask = list(range(num_test))
X_test = X_test[mask]
y_test = y_test[mask]

# Reshape the image data into rows
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
print(X_train.shape, X_test.shape)

(5000, 3072) (500, 3072)


In [5]:
X_train[0].shape

(3072,)

In [6]:
X = np.random.rand(3072)

In [7]:
X.shape

(3072,)

In [8]:
dists_2 = np.zeros(X_train.shape[0])
for j in range(X_train.shape[0]):
   dists_2[j] = (np.sum(np.square(X - X_train[j]))) ** 0.5

dists_2.shape

(5000,)

In [9]:
dists_1 = ((np.square(X_train - X)).sum(axis=1)) ** 0.5

dists_1.shape

(5000,)

In [10]:
dists_1 - dists_2

array([0., 0., 0., ..., 0., 0., 0.])

In [11]:
X_train.shape

(5000, 3072)

In [12]:
X = np.random.rand(10, 3072)
X.shape

(10, 3072)

In [13]:
X_train_2 = (X_train**2).sum(axis=-1)
X_train_2

array([41172800., 63431149., 75888313., ..., 52334313., 75718510.,
       29226892.])

In [14]:
np.tile(X_train_2, (10, 1)).shape

(10, 5000)

In [15]:
(X**2).sum(axis=-1).shape

(10,)

In [16]:
X_2 = (X**2).sum(axis=-1)
X_2

array([1020.37135103, 1014.25084603, 1018.94880331, 1033.65476836,
       1045.80120518, 1010.4543047 , 1031.31739329, 1015.18587433,
       1003.82651659, 1028.0320134 ])

In [17]:
np.tile(X_2, (5000, 1)).shape

(5000, 10)

In [19]:
X_train.shape

(5000, 3072)

In [20]:
y_train.shape

(5000,)

In [26]:
X_train_folds = np.array_split(X_train, 5)
y_train_folds = np.array_split(y_train, 5)

In [47]:
a = list(np.arange(5))
a

[0, 1, 2, 3, 4]

In [48]:
a[1:]

[1, 2, 3, 4]

In [49]:
a.pop(2)

2

In [50]:
a

[0, 1, 3, 4]

In [46]:
for i in range(5):
    print(i)

0
1
2
3
4


In [32]:
Train_1 = np.concatenate(X_train_folds[1:])